In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from time import time
from scipy import sparse
from textwrap import wrap

In [ ]:
def create_small_world(N, lrc_chance):
    # create graph
    G = nx.grid_2d_graph(N, N, periodic=True)
    
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        D_neighbors = np.arange(len(neighbors))
        if neighbors:
            # choose random neighbor to remove edge
            neighbor_to_remove = neighbors[np.random.choice(D_neighbors)]
            # try replace with long range connection
            
            if np.random.random() < lrc_chance:
                # remove the edge between node and neighbor_to_remove
                G.remove_edge(node, neighbor_to_remove)
                # Choose a random node
                D_nodes = np.arange(len(list(G.nodes)))
                random_node = list(G.nodes)[np.random.choice(D_nodes)]
                # Ensure the selected random node is not the same as the current node
                while random_node == node:
                    D_nodes = np.arange(len(list(G.nodes)))
                    random_node = list(G.nodes)[np.random.choice(D_nodes)]
                # Add a long-range connection
                G.add_edge(node, random_node)
    
    return G


In [ ]:
def simulate(G, beta, gamma, start):
    nx.set_node_attributes(G, 0, 'condition')        # add condition (SIR)
    nx.set_node_attributes(G, 0, 'inf_generated')    # add no. of cases generated from node
    nx.set_node_attributes(G, 0, 'time_inf')
    colour_states = {0: 'blue', 1: 'red', 2: 'black'}
    
    if type(list(G.nodes)[0]) != int:
        my_pos = {(x,y):(y,-x) for x,y in G.nodes()}  # display graph as a uniform rectangle (if it's x, y)
        G.nodes[(start[0],start[1])]['condition'] = 1
    else:
        my_pos = None
        G.nodes[(start)]['condition'] = 1

    infected = [x for x,y in G.nodes(data=True) if y['condition']==1]    # list to store infected nodes
    
    # nx.draw(G, pos=my_pos, node_color=[colour_states[node[1]['condition']] 
                                       # for node in G.nodes.data()], node_size=40)
    # plt.show()

    finished = False
    count = 0   # no. of iterations
    
    # lists for graphs later
    susc_list = [len([x for x,y in G.nodes(data=True) if y['condition']==0])]
    inf_list = [len([x for x,y in G.nodes(data=True) if y['condition']==1])]
    rec_list = [len([x for x,y in G.nodes(data=True) if y['condition']==2])]
    time = 0.0    # initial time
    time_list = [0.0]
    
    # R lists
    R1 = [0.0]    # R1 - avg no. of cases per INFECTED individual at time t
    R2 = [0.0]    # R2 - avg no. of cases per RECOVERED individual at time t
    R3 = [0.0]    # R3 - avg no. of cases per individual at time t
    
    # main loop
    while finished == False:
        if len(infected) != 0:
            count += 1
            
            # count number of susceptibles surrounding each infected node and
            # store them in a list the same shape as the infected list
            susceptibles = []
            for i in range(len(infected)):
                no_of_susc = 0
                con_nodes = list(G.adj[infected[i]])
                for j in range(len(con_nodes)):
                    if G.nodes[con_nodes[j]]['condition'] == 0:
                        no_of_susc += 1
                susceptibles.append(no_of_susc)
            
            # apply weights/rates to each infected node, i.e. 
            # rate of infection AND recovery for each node
            susceptibles = np.array(susceptibles)
            susceptibles = (susceptibles * beta) + gamma
            tot_rate = np.sum(susceptibles)
            
            # calculate time elapsed depending on what has happened
            time += 1/tot_rate
            time_list.append(time)    # add to time list for SIR graph later
            
            # turn rates into probabilities
            prob1 = susceptibles/tot_rate
            # choose an infected node randomly (with weights from rates above)
            choice = np.random.choice(np.arange(0, prob1.size), p=prob1)
            
            # get list of all susceptible nodes surrounding infected node
            con_nodes = list(G.adj[infected[choice]])
            con_susc_nodes = []
            for i in range(len(con_nodes)):
                if G.nodes[con_nodes[i]]['condition'] == 0:
                    con_susc_nodes.append(con_nodes[i])
            con_susc_nodes.append(infected[choice])   # add infected node itself to end of list
            
            prob2 = np.zeros(len(con_susc_nodes))
            tot = 0.0
            # loop to create probability array that stores probability of
            # something happening to each surrounding node and initial infected node recovering
            for i in range(len(con_susc_nodes)):
                if G.nodes[con_susc_nodes[i]]['condition'] == 0:
                    prob2[i] = beta
                    tot += beta
                else:
                    prob2[i] = gamma
                    tot += gamma
            prob2 /= tot
            
            # make decision on what to do - analogous to choosing infected node above
            decision = np.random.choice(np.arange(0, prob2.size), p=prob2)
            
            if decision == (prob2.size - 1):
                G.nodes[infected[choice]]['condition'] = 2            # recover
            else:
                G.nodes[con_susc_nodes[decision]]['condition'] = 1    # infect
                G.nodes[infected[choice]]['inf_generated'] += 1       # count case generated by initial infected

            # update infected array, and update infected nodes' time infected
            infected = [x for x,y in G.nodes(data=True) if y['condition']==1]
            for i in range(len(infected)):
                G.nodes[infected[i]]['time_inf'] += 1/tot_rate
            
            # append to lists for SIR graph later
            susc_list.append(len([x for x,y in G.nodes(data=True) if y['condition']==0]))
            inf_list.append(len([x for x,y in G.nodes(data=True) if y['condition']==1]))
            rec_list.append(len([x for x,y in G.nodes(data=True) if y['condition']==2]))
            
            # calculate R's and append to their lists
            # R1 - avg no. of cases per INFECTED individual at time t
            R1_temp = []
            for i in range(len(infected)):
                R1_temp.append(G.nodes[infected[i]]['inf_generated'])
            R1.append(np.mean(R1_temp))
            
            # R2 - avg no. of cases per RECOVERED individual at time t
            R2_temp = []
            recovered = [x for x,y in G.nodes(data=True) if y['condition']==2]
            for i in range(len(recovered)):
                R2_temp.append(G.nodes[recovered[i]]['inf_generated'])
            R2.append(np.mean(R2_temp))
            
            # R3 - avg no. of cases per individual at time t
            everyone = [x for x,y in G.nodes(data=True)]
            R3_temp = []
            for i in range(len(everyone)):
                R3_temp.append(G.nodes[everyone[i]]['inf_generated'])
            R3.append(np.mean(R3_temp))
            
            # display time (and graph) every x iterations
            if count % 500 == 0:
                print('Time: ' + str(time) + '  inf: ' + str(inf_list[-1]) + '  rec: ' + 
                      str(rec_list[-1]))
                # nx.draw(G, pos=my_pos, node_color=[colour_states[node[1]['condition']] 
                                                   # for node in G.nodes.data()], node_size=40)
                # plt.show()
        
        else:
            # nx.draw(G, pos=my_pos, node_color=[colour_states[node[1]['condition']] 
                                                   # for node in G.nodes.data()], node_size=40)
            # plt.show()
            
            finished = True
            break
    
    susc_list = np.array(susc_list)
    inf_list = np.array(inf_list)
    rec_list = np.array(rec_list)
    
    # calculate average time infected
    time_inf_list = []
    everyone = [x for x,y in G.nodes(data=True)]
    for i in range(N):
        time_inf_list.append(G.nodes[everyone[i]]['time_inf'])
    
    tine_inf_list = np.array(time_inf_list)
    avg_time_inf = np.mean(time_inf_list)
    
    # calculate R3(2) - avg. no. of cases per individual (calculated after epidemic)
    R3_2_temp = []
    for i in range(len(everyone)):
        R3_2_temp.append(G.nodes[everyone[i]]['inf_generated'])
    R3_2 = np.mean(R3_2_temp)
    
    return count, time_list, susc_list, inf_list, rec_list, R1, R2, R3, R3_2, avg_time_inf
    

In [ ]:
def plot(time_lists, susc_lists, inf_lists, rec_lists, N, betas, gammas, R1s, R2s, R3s):
    plt.figure(figsize=(9.6,7.2))
    plt.rcParams['text.usetex'] = False
    
    plt.figure()
    plt.plot(time_lists[0], susc_lists[0], label='# of susceptible', color='b')
    plt.plot(time_lists[0], inf_lists[0], label='# of infectious', color='r')
    plt.plot(time_lists[0], rec_lists[0], label='# of recovered', color='black')
    title = "SIR simulation (2D lattice) with " + str(N**2) + " nodes, $γ$ = " + str(np.round(gammas[0], 2)) + ", $β$ = " + str(np.round(betas[0], 2))
    plt.title(title)
    plt.xlabel('Time (days)')
    plt.ylabel('# of nodes')
    plt.legend()
    plt.show()
    
    plt.figure()
    plt.plot(time_lists[1], susc_lists[1], label='# of susceptible', color='b')
    plt.plot(time_lists[1], inf_lists[1], label='# of infectious', color='r')
    plt.plot(time_lists[1], rec_lists[1], label='# of recovered', color='black')
    title = "SIR simulation (small world) with " + str(N**2) + " nodes, $γ$ = " + str(np.round(gammas[1], 2)) + ", $β$ = " + str(np.round(betas[1], 2))
    plt.title(title)
    plt.xlabel('Time (days)')
    plt.ylabel('# of nodes')
    plt.legend()
    plt.show()
    
    plt.figure()
    plt.plot(time_lists[2], susc_lists[2], label='# of susceptible', color='b')
    plt.plot(time_lists[2], inf_lists[2], label='# of infectious', color='r')
    plt.plot(time_lists[2], rec_lists[2], label='# of recovered', color='black')
    title = "SIR simulation (Erdős–Rényi) with " + str(N**2) + " nodes, $γ$ = " + str(np.round(gammas[2], 2)) + ", $β$ = " + str(np.round(betas[2], 2))
    plt.title(title)
    plt.xlabel('Time (days)')
    plt.ylabel('# of nodes')
    plt.legend()
    plt.show()
    
    plt.figure()
    plt.plot(time_lists[0], R1s[0], color='goldenrod', label='2D lattice')
    plt.plot(time_lists[1], R1s[1], color='darkblue', label='Small world')
    plt.plot(time_lists[2], R1s[2], color='darkred', label='Erdős–Rényi')
    title = "\n".join(wrap("Plot of $R_1(t)$ (defined as avg. no. of cases caused by an infected individual at time $t$) vs time", 60))
    plt.title(title)
    plt.xlabel('Time (days)')
    plt.ylabel('$R_1(t)$')
    plt.legend()
    plt.show()
    
    plt.figure()
    plt.plot(time_lists[0], R2s[0], color='goldenrod', label='2D lattice')
    plt.plot(time_lists[1], R2s[1], color='darkblue', label='Small world')
    plt.plot(time_lists[2], R2s[2], color='darkred', label='Erdős–Rényi')
    title = "\n".join(wrap("Plot of $R_2(t)$ (defined as avg. no. of cases caused by a recovered individual up to time $t$) vs time", 60))
    plt.title(title)
    plt.xlabel('Time (days)')
    plt.ylabel('$R_2(t)$')
    plt.legend()
    plt.show()
    
    plt.figure()
    plt.plot(time_lists[0], R3s[0], color='goldenrod', label='2D lattice')
    plt.plot(time_lists[1], R3s[1], color='darkblue', label='Small world')
    plt.plot(time_lists[2], R3s[2], color='darkred', label='Erdős–Rényi')
    title = "\n".join(wrap("Plot of $R_3(t)$ (defined as avg. no. of cases caused by any individual up to time $t$) vs time", 60))
    plt.title(title)
    plt.xlabel('Time (days)')
    plt.ylabel('$R_3(t)$')
    plt.legend()
    plt.show()
    

In [ ]:
def create_graphs(N, lrc_chance, p):
    g_start_time = time()
    G_2D = nx.grid_2d_graph(N, N, periodic=True)
    G_SW = create_small_world(N, lrc_chance)
    G_ER = nx.erdos_renyi_graph(N**2, p)
    g_finish_time = time()
    print('Graphs created in', str(np.round(((g_finish_time - g_start_time)/60), 2)), 'mins.')
    
    print('Check nearest neighbours consistent:')
    print('2D:', '4N^2 =', str(4 * N**2), '; adj. m. =', np.sum(nx.adjacency_matrix(G_2D)))
    print('SW:', '4N^2 =', str(4 * N**2), '; adj. m. =', np.sum(nx.adjacency_matrix(G_SW)))
    print('ER:', '4N^2 =', str(4 * N**2), '; adj. m. =', np.sum(nx.adjacency_matrix(G_ER)))
    
    return G_2D, G_SW, G_ER

def run_full_sim(G_2D, G_SW, G_ER, N, betas, gammas):
    s_start_time = time()
    
    count_2D = 0
    while count_2D < 100:
        count_2D, times_2D, susc_2D, inf_2D, rec_2D, R1_2D, R2_2D, R3_2D, R3_2_2D, ti_2D = simulate(G_2D, betas[0], gammas[0], (N/2, N/2))
    print('2D lattice finished in', count_2D, 'iterations.')
    print()
    
    count_SW = 0
    while count_SW < 100:
        count_SW, times_SW, susc_SW, inf_SW, rec_SW, R1_SW, R2_SW, R3_SW, R3_2_SW, ti_SW = simulate(G_SW, betas[1], gammas[1], (N/2, N/2))
    print('Small world finished in', count_SW, 'iterations.')
    print()
    
    count_ER = 0
    while count_ER < 100:
        count_ER, times_ER, susc_ER, inf_ER, rec_ER, R1_ER, R2_ER, R3_ER, R3_2_ER, ti_ER = simulate(G_ER, betas[2], gammas[2], N/2)
    print('Erdős–Rényi finished in', count_ER, 'iterations.')
    print()
    s_finish_time = time()
    print('---------------------')
    print('Simulations completed in', str(np.round(((s_finish_time - s_start_time)/60), 2)), 'mins.')
          
    total_counts = [count_2D, count_SW, count_ER]
    total_times = [times_2D, times_SW, times_ER]
    total_susc = [susc_2D, susc_SW, susc_ER]
    total_inf = [inf_2D, inf_SW, inf_ER]
    total_rec = [rec_2D, rec_SW, rec_ER]
    total_R1 = [R1_2D, R1_SW, R1_ER]
    total_R2 = [R2_2D, R2_SW, R2_ER]
    total_R3 = [R3_2D, R3_SW, R3_ER]
    total_R3_2 = [R3_2_2D, R3_2_SW, R3_2_ER]
    total_ti = [ti_2D, ti_SW, ti_ER]
          
    plot(total_times, total_susc, total_inf, total_rec, N, betas, gammas, total_R1, total_R2, total_R3)
    print('Mean infection times (2D, SW, ER) =', total_ti[0], total_ti[1], total_ti[2], 'days.')
    print('Calculated R after epidemic (2D, SW, ER) =', total_R3_2[0], total_R3_2[1], total_R3_2[2])
    
    return total_times, total_inf
    

In [ ]:
def get_R4s(total_times, total_inf, gammas):
    total_R4s = []
    total_R4s_orig = []
    
    # smooth I function, and then smooth R_4(t) function
    for i in range(3):
        cs = interpolate.UnivariateSpline(total_times[i], total_inf[i], k=5, s=25000)
        cs_dv = cs.derivative(n=1)
        R4 = cs_dv(total_times[i]) * (1/(total_inf[i] * gammas[i]))
        R4[R4 == -np.inf] = 0.
        R4[R4 == np.inf] = 0.
        total_R4s_orig.append(R4)
        
        R4_cs = interpolate.UnivariateSpline(total_times[i][1:], R4[1:], k=5, s=(R4[1:].size*np.var(R4[1:])))
        total_R4s.append(R4_cs(total_times[i]))
        
        #total_R4s[i][0] = R4[1]
    
    plt.figure(figsize=(12,9))
    
    #plt.plot(total_times[0], total_R4s[0], c='goldenrod', linewidth=2.5, label='2D lattice fit')
    plt.scatter(total_times[0][1:], total_R4s_orig[0][1:], c='khaki', 
                marker='.', s=8, label='2D lattice data', alpha=0.7)
    #plt.plot(total_times[1], total_R4s[1], c='darkblue', linewidth=2.5, label='Small world fit')
    plt.scatter(total_times[1][1:], total_R4s_orig[1][1:], c='cornflowerblue', 
                marker='.', s=8, label='Small world data', alpha=0.7)
    #plt.plot(total_times[2], total_R4s[2], c='darkred', linewidth=2.5, label='Erdős–Rényi fit')
    plt.scatter(total_times[2][1:], total_R4s_orig[2][1:], c='lightsalmon', 
                marker='.', s=8, label='Erdős–Rényi data', alpha=0.7)
    plt.axhline(y=0.0, color='r', linestyle='--', alpha=0.2)
    title = "Plot of $R_0(t)$ (proportional to the rate of infection at time $t$) vs time"
    plt.title(title)
    plt.xlabel('Time (days)')
    plt.ylabel('$R_0(t)$ - 1')
    plt.ylim(-2, 10)
    plt.yticks(np.arange(-2, 11, 1))
    plt.legend()
    plt.show()

In [ ]:
##### CREATE GRAPHS

N = 5
lrc_chance = 0.5    # 0.3
p = 4/(N**2)

G_2D, G_SW, G_ER = create_graphs(N, lrc_chance, p)

In [ ]:
def display_graph(G):
    nx.set_node_attributes(G, 0, 'condition')        # add condition (SIR)
    colour_states = {0: 'blue', 1: 'red', 2: 'black'}

    if type(list(G.nodes)[0]) != int:
        my_pos = {(x,y):(y,-x) for x,y in G.nodes()}  # display graph as a uniform rectangle (if it's x, y)
    else:
        my_pos = None

    nx.draw(G, pos=my_pos, node_size=30)
    plt.show()

In [ ]:
display_graph(G_ER)

In [ ]:
##### RUN FULL SIMULATION

betas = [0.35, 0.35, 0.35]    # betas for 2D, SW, ER
gammas = [1/8, 1/8, 1/8]      # gammas for 2D, SW, ER

total_times, total_inf = run_full_sim(G_2D, G_SW, G_ER, N, betas, gammas)
get_R4s(total_times, total_inf, gammas)